## Authors

* Mihir Rao
* Leah Chowenhill
* Shubham Dubey
* Aniket Sakpal

## Google Drive Mount

In [4]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


## Libraries 

In [5]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from imblearn.under_sampling import RandomUnderSampler
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score, average_precision_score
import warnings

## Load Data From Drive

In [6]:
# unzip -q drive/MyDrive/upfd_data/gossipcop.zip -d gossipcop/
# data_source = "gossipcop" 

!unzip -q drive/MyDrive/upfd_data/politifact.zip -d politifact/
data_source = "politifact" 

In [7]:
data_source = "politifact" 

## Creating the Node Level - Average Out Degree

### Undersatnding-
From pairs which are present in A, we are extracting the information of each pair and storing in processed edge list.
In the end we are taking a count of all the mappings a node might have

In [8]:
data_path = data_source + "/"

with open(data_path + 'A.txt') as f:
    edge_list = f.readlines()

edge_list_processed = list()

only_first_node = list()

for i in range(len(edge_list)):
    each_pair = edge_list[i].replace("\n", "").replace(" ","").split(",")

    each_pair = [int(j) for j in each_pair]

    edge_list_processed.append(each_pair)

    only_first_node.append(each_pair[0])


node_graph_mapping = np.load(data_path +'node_graph_id.npy')
V = len(node_graph_mapping)
edge_list_processed[0]
from collections import Counter 
node_counter = Counter(only_first_node)
len(node_counter.keys())
for i in range(len(node_graph_mapping)):
  if i not in node_counter.keys():
    node_counter[i] = 0

In [9]:
Y = np.load(data_path +'graph_labels.npy')
Y = torch.Tensor(Y).cuda()

In [ ]:
# lbl=np.load(data_path +'graph_labels.npy')

In [ ]:
# lbl.shape

## Node Attributes

In [10]:
# embedding_method = "spacy"

# embedding_method = "content"

embedding_method = "bert"

In [ ]:
# len(np.unique(node_graph_mapping))

In [ ]:
# len(node_graph_mapping)

In [ ]:
# len(edge_list_processed)

In [11]:
source_node_id = np.cumsum(np.bincount(node_graph_mapping))

source_node_id = np.insert(source_node_id, 0, 0)
source_node_id=source_node_id[:len(source_node_id) - 1]

In [ ]:
# source_node_id_test = np.append(0,source_node_id)

In [ ]:
# len(source_node_id)

In [ ]:
# source_node_id

### Undersatnding-
re-shaping the data based on the embedding method

In [ ]:
# embedding_file_path = data_path +"new_" + embedding_method + "_feature.npz"

# new_spacy_feature = np.load(embedding_file_path)
# new_spacy_feature.files
# spacy_data = new_spacy_feature['data']

# if embedding_method == "spacy":
#     spacy_data = spacy_data.reshape(len(node_graph_mapping), 300)
# elif  embedding_method == "bert": 
#     spacy_data = spacy_data.reshape(len(node_graph_mapping), 768)
# elif embedding_method == "content": 
#     spacy_data = spacy_data.reshape(len(node_graph_mapping), 310)
    
# spacy_data.shape

In [ ]:
# spacy_data[0,:20]

In [ ]:
# spacy_data[1,:20]

In [ ]:
# embedding_file_path = data_path +"new_" + embedding_method + "_feature.npz"

# new_spacy_feature = np.load(embedding_file_path)
# new_spacy_feature.files
# spacy_data = new_spacy_feature['data']

# if embedding_method == "spacy":
#     spacy_data = spacy_data.reshape(len(node_graph_mapping), 300)
# elif  embedding_method == "bert": 
#     spacy_data = spacy_data.reshape(len(node_graph_mapping), 768)
# elif embedding_method == "content": 
#     spacy_data = spacy_data.reshape(len(node_graph_mapping), 310)
    
# spacy_data.shape
# # out_degree_nodes  = np.array(list(node_counter.values()))
# # out_degree_nodes.shape
# # out_degree_nodes = out_degree_nodes.reshape(out_degree_nodes.shape[0],1)
# # proc_data = np.append(spacy_data,out_degree_nodes, 1)
# # proc_data.shape
# node_slice = torch.from_numpy(np.bincount(node_graph_mapping))

# node_slice=node_slice.cuda()
# #node_slice = np.array(node_slice)

# proc_data = torch.tensor(spacy_data)

# proc_data = proc_data.cuda()

# node_slice = tuple(node_slice)
# # all_graphs_data = list()
# # for i in node_slice:
# #     # graph_level_data = proc_data[0:i,:]
# #     all_graphs_data.append(list(proc_data[0:i,:].flatten()))

# graph_split_data = list(torch.split(proc_data, node_slice))
# all_graphs_data = list()

In [ ]:
# graph_split_data[0].shape

In [ ]:
# x_testing=torch.mean(graph_split_data[0],0)

In [ ]:
# x_testing.shape

In [12]:
embedding_file_path = data_path +"new_" + embedding_method + "_feature.npz"

new_spacy_feature = np.load(embedding_file_path)
new_spacy_feature.files
spacy_data = new_spacy_feature['data']

if embedding_method == "spacy":
    spacy_data = spacy_data.reshape(len(node_graph_mapping), 300)
elif  embedding_method == "bert": 
    spacy_data = spacy_data.reshape(len(node_graph_mapping), 768)
elif embedding_method == "content": 
    spacy_data = spacy_data.reshape(len(node_graph_mapping), 310)
    
spacy_data.shape
# out_degree_nodes  = np.array(list(node_counter.values()))
# out_degree_nodes.shape
# out_degree_nodes = out_degree_nodes.reshape(out_degree_nodes.shape[0],1)
# proc_data = np.append(spacy_data,out_degree_nodes, 1)
# proc_data.shape
node_slice = torch.from_numpy(np.bincount(node_graph_mapping))

node_slice=node_slice.cuda()
#node_slice = np.array(node_slice)

proc_data = torch.tensor(spacy_data)

proc_data = proc_data.cuda()

node_slice = tuple(node_slice)
# all_graphs_data = list()
# for i in node_slice:
#     # graph_level_data = proc_data[0:i,:]
#     all_graphs_data.append(list(proc_data[0:i,:].flatten()))

graph_split_data = list(torch.split(proc_data, node_slice))
all_graphs_data = list()
for each_graph in graph_split_data:
  # print(graph_split_data)
  a=torch.mean(each_graph[1:],0)
  b=each_graph[0]
  c=torch.cat((a,b),dim=0)
  all_graphs_data.append(c)
    
all_graphs_data_padded = pad_sequence(all_graphs_data, batch_first= True) 
X = all_graphs_data_padded

In [13]:
X.shape

torch.Size([314, 1536])

In [14]:
## Check
check = list()
for i in all_graphs_data_padded:
  check.append(len(i))

if embedding_method == "spacy":
    list(set(check))[0]/301
else:
    list(set(check))[0]/769
  
len(all_graphs_data_padded[0])
all_graphs_data_padded[1]

tensor([ 0.0244,  0.0328,  0.0164,  ..., -0.0707, -0.0849,  0.5245],
       device='cuda:0', dtype=torch.float64)

In [15]:
all_graphs_data_padded.shape


train_len=int(0.7*len(X))
val_len=int(0.2*len(X))
test_len=len(X)-val_len-train_len
train_len,val_len,test_len
mn=np.split(np.arange(5464),(train_len,val_len,test_len))
mn_x=torch.split(X,(train_len,val_len,test_len))
mn_y=torch.split(Y,(train_len,val_len,test_len))

for x in mn_x:
  print(x.shape)


torch.Size([219, 1536])
torch.Size([62, 1536])
torch.Size([33, 1536])


## Split into Train, Validation, Test

In [16]:

np.random.seed(17)
a=np.arange(X.shape[0])
np.random.shuffle(a)



train_len=int(0.7*len(X))
val_len=int(0.2*len(X))
test_len=len(X)-val_len-train_len
train_len,val_len,test_len
#mn=np.split(np.random.shuffle(np.arange(5464)),(train_len,val_len,test_len))
X=X[a]
Y=Y[a]
mn_x=torch.split(X,(train_len,val_len,test_len))
mn_y=torch.split(Y,(train_len,val_len,test_len))

for x in mn_x:
  print(x.shape)

torch.Size([219, 1536])
torch.Size([62, 1536])
torch.Size([33, 1536])


In [17]:
num_training = int(len(X) * 0.2)
num_val = int(len(X) * 0.1)
num_test = len(X) - (num_training + num_val)

#x_train, x_val, y_train, y_val = train_test_split( X, Y, test_size = 0.3 ,random_state=17 )
#x_train, x_val, y_train, y_val =X[mn[0]],X[mn[1]],Y[mn[0]],Y[mn[1]]

x_train, x_val, x_test=mn_x[0],mn_x[1], mn_x[2]

y_train, y_val, y_test=mn_y[0],mn_y[1], mn_y[2]

x_test=mn_x[2]


#x_val , x_test , y_val , y_test = train_test_split(x_val, y_val, test_size = 0.33, random_state=17)

#x_test,y_test=X[mn[2]],Y[mn[2]]

In [18]:
print(x_train.shape,x_val.shape,x_test.shape)
y_train.shape,y_val.shape,y_test.shape



torch.Size([219, 1536]) torch.Size([62, 1536]) torch.Size([33, 1536])


(torch.Size([219]), torch.Size([62]), torch.Size([33]))

In [ ]:
# y_train[:100]

## Dataloader 

In [19]:
class Fetch_Rows(torch.utils.data.Dataset):
  def __init__(self, X, Y = None, only_test = False):
    self.only_test = only_test

    self.X = X
    
    if only_test==False:
       self.Y = Y
    
    self.length = len(X)

 
  def __len__(self):
      return self.length
        
  def __getitem__(self, i):

      xx = self.X[i]
      
      if self.only_test == False:
          yy = self.Y[i]   

      if self.only_test == False:
        return xx, yy
      else:
        return xx
    

## Network

### 4 layered MLP - inverted cone

In [20]:
class Network(torch.nn.Module):
    def __init__(self, in_size):
        super(Network, self).__init__()

        layers = [
            nn.Linear(in_size,100),
            nn.Linear(100, 50),
            nn.Linear(50,25),
            nn.Linear(25,2)
        ]

        self.laysers = nn.Sequential(*layers)
       

    def forward(self, A0):
        x = self.laysers(A0)


        return x

## Train & Validation & Test Configuration

In [21]:
# def train(model, train_loader, optimizer, criterion):
#     model.train()
    
#     total_loss = 0

#     for i, data in (enumerate(train_loader)):
        
#         optimizer.zero_grad()

#         x, y = data
#         x = x.float().cuda()
#         y = y.long().cuda()

#         output= model(x)

#         # print(x)
#         # print(type(x))
#         # print(type(output))
#         # print(output)
#         # print(y)

#         loss = criterion(output, y)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()
      
#     avg_loss = total_loss/(i+1)

#     return model, avg_loss

def train(model, train_loader, optimizer, criterion):
    model.train()
    
    total_loss = 0

    for i, data in (enumerate(train_loader)):
        
        optimizer.zero_grad()

        x, y = data
        x = x.float().cuda()
        y = y.long().cuda()

        output= model(x)

        # print(x)
        # print(type(x))
        # print(type(output))
        # print(output)
        # print(y)

        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
    return model,loss.item()



def validation( model, device, val_loader):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
      for data, true_y in val_loader:
          data = data.float().to(device)
          true_y = true_y.long().to(device)                
          
          output = model(data)

          pred_y = torch.argmax(output, axis=1)

          pred_y_list.extend(pred_y.tolist())
          
          true_y_list.extend(true_y.tolist())

    val_accuracy =  accuracy_score(true_y_list, pred_y_list)
    accuracy = accuracy_score(true_y_list, pred_y_list)
    f1_macro = f1_score(true_y_list, pred_y_list, average='macro') 
    f1_micro = f1_score(true_y_list, pred_y_list, average='micro') 
    precision = precision_score(true_y_list, pred_y_list, zero_division=0)
    recall = recall_score(true_y_list, pred_y_list, zero_division=0)

    return val_accuracy, f1_macro, f1_micro, precision, recall    

In [ ]:
# def validation( model, device, val_loader):
#     model.eval()
#     true_y_list = []
#     pred_y_list = []
#     with torch.no_grad():
#       for data, true_y in val_loader:
#           data = data.float().to(device)
#           true_y = true_y.long().to(device)                
          
#           output = model(data)

#           pred_y = torch.argmax(output, axis=1)

#           pred_y_list.extend(pred_y.tolist())
          
#           true_y_list.extend(true_y.tolist())

#     val_accuracy =  accuracy_score(true_y_list, pred_y_list)
#     accuracy = accuracy_score(true_y_list, pred_y_list)
#     f1_macro = f1_score(true_y_list, pred_y_list, average='macro') 
#     f1_micro = f1_score(true_y_list, pred_y_list, average='micro') 
#     precision = precision_score(true_y_list, pred_y_list, zero_division=0)
#     recall = recall_score(true_y_list, pred_y_list, zero_division=0)

#     return val_accuracy, f1_macro, f1_micro, precision, recall

In [22]:
def test( model, device, test_loader,y_test=None,my_dataset=True,):
    model.eval()
    y_test1 = []
    pred_y_list = []
    with torch.no_grad():
      for data_x in test_loader:
          
          data_x = data_x.float().to(device)               
          
          output = model(data_x)

          pred_y = torch.argmax(output, axis=1)

          pred_y_list.extend(pred_y.tolist())
          
    if my_dataset:
      y_test = y_test.long().to(device) 
      y_test1.extend(y_test.tolist())
      val_accuracy =  accuracy_score(y_test1, pred_y_list)
      accuracy = accuracy_score(y_test1, pred_y_list)
      f1_macro = f1_score(y_test1, pred_y_list, average='macro') 
      f1_micro = f1_score(y_test1, pred_y_list, average='micro') 
      precision = precision_score(y_test1, pred_y_list, zero_division=0)
      recall = recall_score(y_test1, pred_y_list, zero_division=0)
      # print(confusion_matrix(y_test1, pred_y_list))
      # round(answer, 2)

    return round(val_accuracy,2), round(f1_macro,2), round(f1_micro,2), round(precision,2), round(recall,2)

## Training 

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network(in_size = len(all_graphs_data_padded[0])).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
scheduler=StepLR(optimizer,step_size=1,gamma=0.2)

In [24]:
train_items = Fetch_Rows(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_items,shuffle=True  )

val_items = Fetch_Rows(x_val, y_val)
val_loader = torch.utils.data.DataLoader(val_items,shuffle=False)

test_items = Fetch_Rows(x_test, y_test,only_test=True)
test_loader = torch.utils.data.DataLoader(test_items, shuffle=False)


for epoch in range(1, 30):
    model ,avg_loss = train( model,  train_loader, optimizer, criterion)
    
    val_accuracy, f1_macro, f1_micro, precision, recall = validation(model, device, val_loader)

    print('Epoch',epoch," Training Loss ",avg_loss , "Validation Accuracy ", val_accuracy, "F1 Macro ", f1_macro, "F1 Micro ",f1_micro, "Precision ", precision, "Recall ", recall)

Epoch 1  Training Loss  0.21678507328033447 Validation Accuracy  0.5 F1 Macro  0.4202111613876319 F1 Micro  0.5 Precision  0.46551724137931033 Recall  1.0
Epoch 2  Training Loss  0.13608413934707642 Validation Accuracy  0.6935483870967742 F1 Macro  0.6895915678524375 F1 Micro  0.6935483870967742 Precision  0.5952380952380952 Recall  0.9259259259259259
Epoch 3  Training Loss  0.8194882273674011 Validation Accuracy  0.5645161290322581 F1 Macro  0.5193798449612402 F1 Micro  0.5645161290322581 Precision  0.5 Recall  1.0
Epoch 4  Training Loss  0.28896364569664 Validation Accuracy  0.7741935483870968 F1 Macro  0.7732497387669801 F1 Micro  0.7741935483870968 Precision  0.6666666666666666 Recall  0.9629629629629629
Epoch 5  Training Loss  6.19869097135961e-05 Validation Accuracy  0.8064516129032258 F1 Macro  0.7926421404682275 F1 Micro  0.8064516129032258 Precision  0.8947368421052632 Recall  0.6296296296296297
Epoch 6  Training Loss  0.28098469972610474 Validation Accuracy  0.822580645161290

## Test

In [25]:
val_accuracy, f1_macro, f1_micro, precision, recall=test(model,device,test_loader,y_test,my_dataset=True)
print("Test Accuracy ", val_accuracy, "\nF1 Macro ", f1_macro, "\nF1 Micro ",f1_micro, "\nPrecision ", precision, "\nRecall ", recall)


Test Accuracy  0.97 
F1 Macro  0.97 
F1 Micro  0.97 
Precision  1.0 
Recall  0.94
